In [1]:
# !pip install prometheus-api-client

In [1]:
import pandas as pd
from prometheus_api_client import PrometheusConnect
from prometheus_api_client.utils import parse_datetime

prometheus_url = "http://prometheus-server.monitoring:80"

prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)

query = """
sum(
  (
	sum(
		sum(container_memory_allocation_bytes{job="kubecost"}) by (namespace,instance) * on(instance) group_left() (
				node_ram_hourly_cost{job="kubecost"} / 1024 / 1024 / 1024 * 730
				+ on(node,instance_type) group_left()
					label_replace
					(
						kube_node_labels{job="kubecost"}, "instance_type", "$1", "label_node_kubernetes_io_instance_type", "(.*)"
					) * 0
			)
  		+ 
  		sum(container_cpu_allocation{job="kubecost"}) by (namespace,instance) * on(instance) group_left() (
	  			node_cpu_hourly_cost{job="kubecost"} + on(node,instance_type) group_left()
		  			label_replace
		  			(
		  				kube_node_labels{job="kubecost"}, "instance_type", "$1", "label_node_kubernetes_io_instance_type", "(.*)"
		  			) * 0
		  	) * 730) by (namespace)
	) and on(namespace) kube_namespace_labels{label_billable="true", job="kubecost"}
) by (namespace)
"""

start_time = '2024-02-01T21:00:00'
end_time = '2024-02-02T21:00:00'

start_time = datetime.fromisoformat(start_time)
end_time = datetime.fromisoformat(end_time)

#start_time = parse_datetime('1d')
#end_time = parse_datetime('now')

step_in_sec = 60

# Fazer a consulta
result = prom.custom_query_range(query, start_time, end_time, step_in_sec)

df = pd.DataFrame(result)

df['namespace'] = df['metric'].apply(lambda x: x['namespace'])
df['time_extraction'] = df['values'].apply(lambda x: x[0][0])
df['porcentagem'] = df['values'].apply(lambda x: sum(float(pair[1]) for pair in x))

df['time_extraction'] = pd.to_datetime(df['time_extraction'], unit='s')
df['year'] = df['time_extraction'].dt.year
df['month'] = df['time_extraction'].dt.month
df['day'] = df['time_extraction'].dt.day

df['Porcentagem Normalizada'] = 100* (df['porcentagem'] / df['porcentagem'].sum())

df = df[['namespace', 'Porcentagem Normalizada', 'time_extraction', 'year', 'month', 'day']]
df = df.sort_values(by='Porcentagem Normalizada', ascending=False)

df.head(20)

NameError: name 'datetime' is not defined

In [155]:
df['Porcentagem Normalizada'].sum()

99.99999999999999

# Namespace Labels

In [ ]:
import pandas as pd
from prometheus_api_client import PrometheusConnect

prometheus_url = "http://prometheus-server.monitoring:80"

prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)

query = """kube_namespace_labels{job="kubecost"}"""

data = prom.custom_query(query)

metrics = [metric['metric'] for metric in data]

df = pd.DataFrame(metrics)

df = df[['namespace', 
         'label_acr_secret', 
         'label_billable', 
         'label_environment', 
         'label_project', 
         'label_team', 
         'label_spot', 
         'label_airflow']]

# Exibir o DataFrame
df.sort_values('namespace')

filtered_df = df[df['label_billable'] == 'true']

filtered_df.head(20)

: 

# Pod Label

In [ ]:
import pandas as pd
from prometheus_api_client import PrometheusConnect

prometheus_url = "http://prometheus-server.monitoring:80"

prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)

query = """kube_pod_labels{job="kubecost"}"""

data = prom.custom_query(query)

metrics = [metric['metric'] for metric in data]

df = pd.DataFrame(metrics)

df = df[['label_dag_id', 
         'namespace']]

# Exibir o DataFrame
df.sort_values('namespace')

filtered_df = df[df['label_billable'] == 'true']

filtered_df.head(20)